In [7]:
jupyter_pwd = %pwd
if jupyter_pwd == "/":
    %cd /workspace

# %pdb on

# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# ipynb形式のライブラリノートを.py形式に変更したものをインポート
import lib
import lib.lab_lib
from lib.lab_lib import *

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# NPBのプロセス数
npb_process :list[int] = [2, 4, 8, 16, 32, 64, 128, 256]
train_npb_process :list[int] = npb_process[:-1]
test_npb_process :list[int] = npb_process[-1:]
# NPBのCGの初期変数
cg_na: list[int] = [14000, 30000, 75000, 100000, 1500000]
cg_nonzer: list[int] = [11, 12, 13, 14, 15, 18, 21]
cg_niter: list[int] = [15, 30, 75, 90, 100]
cg_shift: list[int] = [20, 40, 60, 80, 110, 200]

train_cg_na: list[int] = cg_na[:-1]
train_cg_nonzer: list[int] = cg_nonzer[:-1]
train_cg_niter: list[int] = cg_niter[:-1]
train_cg_shift: list[int] = cg_shift[:-1]

test_cg_na: list[int] = cg_na[-1:]
test_cg_nonzer: list[int] = cg_nonzer[-1:]
test_cg_niter: list[int] = cg_niter[-1:]
test_cg_shift: list[int] = cg_shift[-1:]

# NPBのEPの初期変数
train_ep_process :list[int] = [4, 8, 16, 32, 64]
train_ep_size :list[str] = ["S", "W", "A", "B", "C"]
test_ep_process :list[int] = [128, 256, 512]
test_ep_size :list[str] = ["D", "E", "F"]

# NPBのFTの初期変数
train_ft_process :list[int] = [2, 4, 8, 16, 32, 64]
train_ft_grid_size :list[int] = [32, 64, 128, 256, 512]
train_ft_nit :list[int] = [5, 10, 15, 20, 25]

test_ft_process :list[int] = [128, 256, 512]
test_ft_grid_size :list[int] = [1024, 2048, 4096]
test_ft_nit :list[int] = [30, 40, 50]

# NPBのISの初期変数
train_is_process :list[int] = [2, 4, 8, 16]
train_is_no_of_keys :list[int] = [18, 20, 22, 24, 26]
train_is_max_value :list[int] = [9, 11, 13, 15]

test_is_process :list[int] = [32, 64, 128]
test_is_no_of_keys :list[int] = [28, 29, 30]
test_is_max_value :list[int] = [18, 19, 20]

# NPBのMGの初期変数
mg_size :list[int] = [32, 64, 128, 256, 512]
mg_nit: list[int] = [4, 10, 20, 35, 50]

train_mg_process :list[int] = [4, 8, 16, 32, 64]
train_mg_size :list[int] = [4, 8, 16, 32, 64]
train_mg_nit :list[int] = [5, 10, 15, 20, 25]

test_mg_process :list[int] = [128, 256, 512]
test_mg_size :list[int] = [128, 256, 512]
test_mg_nit :list[int] = [30, 40, 50]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

train_lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343]
train_lulesh_iterations: list[int] = [8, 16, 32, 64, 128]
train_lulesh_sizes: list[int] = [16, 24, 32, 48]

test_lulesh_processes: list[int] = [512, 729, 1000]
test_lulesh_iterations: list[int] = [256, 512, 1024]
test_lulesh_sizes: list[int] = [64, 96, 128]

# Extra-Pのオプション
modelerNames: list[str] = [
    # "refining", 
    "multi-parameter",
    "default", 
    # "basic --options poly_exponents=-1,0,1,2,3 log_exponents=0,1 force_combination_exponents=1 allow_negative_exponents=1"
    ]

modelerOption: str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""

list_csvDir = [
    "./csv_files/lulesh_1st/",
    "./csv_files/lulesh_2nd/",
    "./csv_files/lulesh_3rd/",
]

list_csvDir_mg = [
    "./csv_files/mg_1st/",
    "./csv_files/mg_2nd/",
    "./csv_files/mg_3rd/",
]

list_csvDir_ft :list[str] = [
    "./csv_files/ft_1st/",
    "./csv_files/ft_2nd/",
    "./csv_files/ft_3rd/",
]

list_csvDir_ep :list[str] = [
    "./csv_files/ep_1st/",
    "./csv_files/ep_2nd/",
    "./csv_files/ep_3rd/",
]

list_csvDir_is :list[str] = [
    "./csv_files/is_1st/",
    "./csv_files/is_2nd/",
    "./csv_files/is_3rd/",
]

DEBUG:__main__:hello


In [8]:
def return_rawDF_is(
    list_process: list[int],
    list_no_of_keys: list[int],
    list_key_max_value :list[int],
    csvDir: str,
) -> pd.DataFrame:
    """return_rawDF_is()

    ベンチマークプログラムISの手動で変更した初期変数におけるプロファイルを取得する関数

    Args:
        list_process(list[int]):プロセス数のリスト
        list_no_of_keys(list[int]):初期変数no_of_keysのリスト
        list_key_max_value(list[int]):初期変数key_max_valueのリスト
        csvDir(str):CSVファイルの保持されているディレクトリ

    Returns:
        pd.DataFrame

    """

    list_before_concat_DF: list[pd.DataFrame] = []

    for elem_process in list_process:
        for elem_no_of_keys in list_no_of_keys:
            for elem_key_max_value in list_key_max_value:
                filePath: str = f"{csvDir}is_no_of_keys{elem_no_of_keys}_key_max_value{elem_key_max_value}_process{elem_process}.csv"
                if os.path.isfile(filePath):
                    try:
                        DF_read_raw: pd.DataFrame = pd.read_csv(filePath)
                        DF_read_raw["process"] = elem_process
                        DF_read_raw["no_of_keys"] = elem_no_of_keys
                        DF_read_raw["key_max_value"] = elem_key_max_value
                        list_before_concat_DF.append(DF_read_raw)
                    except:
                        warnings.warn(f"{filePath} is empty.")
                else:
                    warnings.warn(f"{filePath} doesn't exist")
                    continue
    return pd.concat(objs=list_before_concat_DF, axis=0)


def ret_averaged_rawDF_is(
    list_process: list[int],
    list_no_of_keys: list[int],
    list_key_max_value :list[int],
    list_csvDir: list[str],
    resVar: str,
):
    """複数のCSVからDFを取得する関数（ベンチマークプログラムEP）

    列Inclusiveおよび列Exclusiveが秒に変換され、InclusivePerCallもしくはExclusivePerCall列が生成される。

    Args:
        list_process(list[int]):プロセス数のリスト
        list_no_of_keys(list[int]):初期変数no_of_keysのリスト
        list_key_max_value(list[int]):初期変数key_max_valueのリスト
        list_csvDir(list[str]):CSVを保持したディレクトリ名のリスト
        resVar(str):説明変数の文字列

    """

    list_DFs_for_return: list[pd.DataFrame] = []
    for elem_process in list_process:
        for elem_no_of_keys in list_no_of_keys:
            for elem_key_max_value in list_key_max_value:
                list_inputDFs_for_averaged: list[pd.DataFrame] = []
                for elem_csvDir in list_csvDir:
                    try:
                        _raw_DF: pd.DataFrame = return_rawDF_is(
                            list_process=[elem_process],
                            list_no_of_keys=[elem_no_of_keys],
                            list_key_max_value=[elem_key_max_value],
                            csvDir=elem_csvDir,
                        )

                        if resVar in ["Exclusive", "Inclusive", "#Call", "#Subrs"]:
                            # resVar 列の整形
                            if resVar in ["Exclusive", "Inclusive"]:
                                _tmp_converted = map(
                                    convertPprofTime, list(_raw_DF[resVar])
                                )
                                _raw_DF[resVar] = list(_tmp_converted)
                            # {resVar}PerCall 列の生成
                            _raw_DF = add_perCallColumn(
                                inputDF=_raw_DF,
                                divisorColName="#Call",
                                dividendColName=resVar,
                                targetColumnName=f"{resVar}PerCall",
                            )
                    except:
                        continue

                    list_inputDFs_for_averaged.append(_raw_DF)
                list_DFs_for_return.append(
                    ret_averagedDF(inputDFs=list_inputDFs_for_averaged, resVar=resVar)
                )
    return pd.concat(objs=list_DFs_for_return, axis=0)

In [9]:
trainDF_FT :pd.DataFrame = ret_averaged_rawDF_ft(list_process=train_ft_process, list_grid_size=train_ft_grid_size, list_nit=train_ft_nit, list_csvDir=list_csvDir_ft, resVar="Exclusive")


testDF_FT :pd.DataFrame = ret_averaged_rawDF_ft(list_process=test_ft_process, list_grid_size=test_ft_grid_size, list_nit=test_ft_nit, list_csvDir=list_csvDir_ft, resVar="Exclusive")


trainDF_MG :pd.DataFrame = ret_averaged_rawDF_mg(list_process=train_mg_process, list_size=train_mg_size, list_nit=train_mg_nit, list_csvDir=list_csvDir_mg, resVar="Exclusive")


testDF_MG :pd.DataFrame = ret_averaged_rawDF_mg(list_process=test_mg_process, list_size=test_mg_size, list_nit=test_mg_nit, list_csvDir=list_csvDir_mg, resVar="Exclusive")


trainDF_EP :pd.DataFrame = ret_averaged_rawDF_ep(list_process=train_ep_process, list_size=train_ep_size, list_csvDir=list_csvDir_ep, resVar="Exclusive")


testDF_EP :pd.DataFrame = ret_averaged_rawDF_ep(list_process=test_ep_process, list_size=test_ep_size, list_csvDir=list_csvDir_ep, resVar="Exclusive")


trainDF_IS :pd.DataFrame = ret_averaged_rawDF_is(list_process=train_is_process, list_key_max_value=train_is_max_value, list_no_of_keys=train_is_no_of_keys, list_csvDir=list_csvDir_is, resVar="Exclusive")


testDF_IS :pd.DataFrame = ret_averaged_rawDF_is(list_process=test_is_process, list_key_max_value=test_is_max_value, list_no_of_keys=test_is_no_of_keys, list_csvDir=list_csvDir_is, resVar="Exclusive")

expVars_ep :list[str] = ["process", "size"]
expVars_ft :list[str] = ["process", "grid_size", "nit"]
expVars_is :list[str] = ["process", "no_of_keys", "key_max_value"]
expVars_mg :list[str] = ["process", "problem_size", "nit"]

In [10]:

print(testDF_EP.head())
print(testDF_IS.head())


   %Time  Exclusive Inclusive   #Call    #Subrs              Name  process  \
0  100.0   0.000006    54,584     1.0       1.0  .TAU_application      128   
1  100.0  24.626000    54,584     1.0  237605.0             EMBAR      128   
2   49.2   9.965000    26,845     4.0       0.0   MPI_Allreduce()      128   
3    2.8   1.073333     1,536  8194.0       0.0            VRANLC      128   
4    1.4   0.656000       747     1.0       0.0        MPI_Init()      128   

  size  ExclusivePerCall  
0    D          0.000006  
1    D         24.912000  
2    D          6.711250  
3    D          0.000187  
4    D          0.747000  
   %Time  Exclusive Inclusive       #Call        #Subrs  \
0  100.0   0.000007    11,173         1.0  1.000000e+00   
1  100.0   0.000095    11,173         1.0  3.306250e+01   
2   73.6   3.485000     8,218         1.0  3.355440e+07   
3   42.3   4.719667     4,728  33554500.0  0.000000e+00   
4   10.6   0.872667     1,189        11.0  3.300000e+01   

              

# 必要な結果

* 関数コール回数予測
    * モデル適合度(MAPE表)
    * モデル予測精度(MAPEが縦軸のグラフ、重み付きMAPEが縦軸のグラフ)
* 実行時間予測
    * モデル適合度(MAPE表)
    * モデル予測精度(MAPEが縦軸のグラフ、重み付きMAPEが縦軸のグラフ)

# 必要なブツ

* ✅モデル構築用データ
* ✅予測対象用データ
* ✅関数コール回数の予測モデル
* ✅関数の総実行時間の予測モデル
* ✅1コール当たりの関数の総実行時間の予測モデル


In [11]:
class ContentsForExtraP:
    """class ContentsForExtraP

    ExtraPによるモデルの生成

    生成されるモデルは次の通り。

    * 関数コール回数の予測モデル
    * 関数の総実行時間の予測モデル
    * 1コール当たりの関数の総実行時間の予測モデル

    """

    def __init__(
        self,
        trainDF: pd.DataFrame,
        testDF: pd.DataFrame,
        expVars: list[str],
        resVar: str,
        resVarPerCall: str,
        benchmarkName :str,
        modelerName :str = "multi-parameter",
        modelerOption :str = """ --options \#spm=Basic \#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True"""
    ):
        """__init__()

        初期化変数

        Args:
            trainDF(pd.DataFrame):モデル構築用データ
            testDF(pd.DataFrame):予測対象用データ
            expVars(list[str]):説明変数を格納したリスト
            resVar(str):目的変数を示した文字列
            resVarsPerCall(str):1コール当たりの目的変数を示した文字列

        """
        self.trainDF = trainDF.reset_index()
        self.testDF = testDF.reset_index()
        self.expVars = expVars
        self.resVar = resVar
        self.resVarPerCall = resVarPerCall
        self.functionNames :list[str] = sorted(list(set(trainDF["Name"].to_list())))
        self.benchmarkName :str = benchmarkName
        self.modelerName :str = modelerName
        self.modelerOption :str = modelerOption
        self.dict_symbols :dict[str, str] = {}
        for elem in expVars:
            self.dict_symbols[elem] = symbols(elem, real=True)
        
        self.dict_resVar = {}
        self.dict_resVarPerCall = {}
        self.dict_call = {}

    def build_all_models(self):
        """build_all_models()

        モデルを構築する関数。目的変数はself.resVars, self.resVarsPerCallを利用。

        """

        for functionName in self.functionNames:
            trainDF_perFunc :pd.DataFrame = self.trainDF[self.trainDF["Name"] == functionName]

            model_resVar :str = get_ExtraP_model(
                inputDF_perFunc=trainDF_perFunc,
                expVar = self.expVars,
                resVar = self.resVar,
                functionName = functionName,
                dict_symbols=self.dict_symbols,
                benchmarkName = self.benchmarkName,
                modelerName = self.modelerName,
                modelerOption = self.modelerOption
            )

            model_resVarPerCall :str = get_ExtraP_model(
                inputDF_perFunc=trainDF_perFunc,
                expVar = self.expVars,
                resVar = self.resVarPerCall,
                functionName = functionName,
                dict_symbols=self.dict_symbols,
                benchmarkName = self.benchmarkName,
                modelerName = self.modelerName,
                modelerOption = self.modelerOption
            )

            model_call :str = get_ExtraP_model(
                inputDF_perFunc=trainDF_perFunc,
                expVar = self.expVars,
                resVar = "#Call",
                functionName = functionName,
                dict_symbols=self.dict_symbols,
                benchmarkName = self.benchmarkName,
                modelerName = self.modelerName,
                modelerOption = self.modelerOption
            )

            self.dict_resVar[functionName] = model_resVar
            self.dict_resVarPerCall[functionName] = model_resVarPerCall
            self.dict_call[functionName] = model_call

    def predict_train(self):
        """predict_train(self)

        学習データに対して予測を実施する関数

        列構成は下記の通り

        |<expVar>|Name(関数名)|#Call(コール回数実測値)|predicted_resVar（）|predicted_ExclusivePerCall	predicted_#Call>|<>|

        """

        _list_series :list[pd.Series] = []
        for index, _sr in self.trainDF.iterrows():
            _series :pd.Series = pd.Series(dtype="object")
            functionName :str = _sr["Name"]

            _target_env :list[set[any]] = []
            for _expVar in self.expVars:
                _series[_expVar] = _sr[_expVar]
                _target_env.append((self.dict_symbols[_expVar], _sr[_expVar]))

            _series["Name"] = _sr["Name"]
            _series["#Call"] = _sr["#Call"]
            _series[f"{self.resVar}"] = _sr[f"{self.resVar}"]

            _predicted_resVar: float
            _predicted_resVarPerCall :float
            _predicted_call :float

            _predicted_resVar = self.dict_resVar[functionName].subs(_target_env).evalf()
            _predicted_resVarPerCall = self.dict_resVarPerCall[functionName].subs(_target_env).evalf()
            _predicted_call = self.dict_call[functionName].subs(_target_env).evalf()

            _series[f"predicted_{self.resVar}"] = _predicted_resVar
            _series[f"predicted_{self.resVarPerCall}"] = _predicted_resVarPerCall
            _series[f"predicted_#Call"] = _predicted_call
            _series[f"predicted_{self.resVar}_indirectly"] = _predicted_resVarPerCall * _predicted_call

            _list_series.append(_series)

        self.DF_predicted_by_train :pd.DataFrame = pd.DataFrame(data=_list_series)

    def predict_test(self):
        """predict_test(self)
        
        予測対象データに対して予測を実施する関数
        """

        _list_series :list[pd.Series] = []

        for index, _sr in self.testDF.iterrows():
            _series :pd.Series = pd.Series(dtype="object")
            functionName :str = _sr["Name"]

            _target_env :list[set[any]] = []
            for _expVar in self.expVars:
                _series[_expVar] = _sr[_expVar]
                _target_env.append((self.dict_symbols[_expVar], _sr[_expVar]))

            _series["Name"] = functionName
            _series["#Call"] = _sr["#Call"]
            _series[f"{self.resVar}"] = _sr[f"{self.resVar}"]

            _predicted_resVar :float = self.dict_resVar[functionName].subs(_target_env).evalf()
            _predicted_resVarPerCall :float = self.dict_resVarPerCall[functionName].subs(_target_env).evalf()
            _predicted_call :float = self.dict_call[functionName].subs(_target_env).evalf()

            _series[f"predicted_{self.resVar}"] = _predicted_resVar
            _series[f"predicted_{self.resVarPerCall}"] = _predicted_resVarPerCall
            _series[f"predicted_#Call"] = _predicted_call
            _series[f"predicted_{self.resVar}_indirectly"] = _predicted_resVarPerCall * _predicted_call

            _list_series.append(_series)

        self.DF_predicted_by_test :pd.DataFrame = pd.DataFrame(data=_list_series)

    def get_result_of_predict_train(self):
        return self.DF_predicted_by_train

    def get_DF_all_fitness_with_relativeErrorRate(self) -> pd.DataFrame:
        """get_DF_all_fitness_with_relativeErrorRate()
        
        モデル適合度のテーブルを返す関数

        列APE～の平均をとるとMAPEになる。

        """

        _returnDF :pd.DataFrame = add_relativeErrorRateCol(
            inputDF = self.DF_predicted_by_train,
            real_colName="#Call",
            predicted_colName="predicted_#Call",
            targetColName="APE_#Call"
        )
        _returnDF = add_relativeErrorRateCol(
            inputDF = _returnDF,
            real_colName=f"{self.resVar}",
            predicted_colName=f"predicted_{self.resVar}",
            targetColName=f"APE_{self.resVar}",
        )
        _returnDF = add_relativeErrorRateCol(
            inputDF = _returnDF,
            real_colName=f"{self.resVar}",
            predicted_colName=f"predicted_{self.resVar}_indirectly",
            targetColName=f"APE_{self.resVarPerCall}",
        )

        return _returnDF

    def get_DF_weightedMAPE(self) -> pd.DataFrame:
        """get_DF_all_fitness_with_weightedMAPE()"""
        _dict_expVar_set :dict[str, set[int]]= {}
        for _expVar in self.expVars:
            _dict_expVar_set[_expVar] = set(sorted(self.testDF[_expVar]))

        if len(self.expVars) == 2:
            _product = itertools.product(
                _dict_expVar_set[self.expVars[0]],
                _dict_expVar_set[self.expVars[1]]
            )
        elif len(self.expVars) == 3:
            _product = itertools.product(
                _dict_expVar_set[self.expVars[0]],
                _dict_expVar_set[self.expVars[1]],
                _dict_expVar_set[self.expVars[2]]
            )
        else:
            warnings.warn(f"self.expVars == {len(self.expVars)}")
            return -1

        def ret_query (list_expVar :list[str], list_product :list[int]) -> str:
            retStr :str= ""
            for i_expVar in range(len(list_expVar)):
                retStr += f"{list_expVar[i_expVar]}=={list_product[i_expVar]}"
                if i_expVar != len(list_expVar)-1:
                    retStr += " & "
            return retStr

        _list_series :pd.Series = []
        for _elem_product in _product:

            _query :str = ret_query(list_expVar = self.expVars, list_product=list(_elem_product))

            _targetDF :pd.DataFrame = self.testDF.query(_query)
            _target_env :list[set[any]] = []

            _series :pd.Series = pd.Series(dtype="object")

            for i_expVar in range(len(self.expVars)):
                _target_env.append((self.dict_symbols[self.expVars[i_expVar]], _elem_product[i_expVar]))
                _series[self.expVars[i_expVar]] = _elem_product[i_expVar]

            _c_sum :float = sum(_targetDF["#Call"].tolist())
            _target_weightedMAPE_resVar :float = 0.0
            _target_weightedMAPE_resVarPerCall :float = 0.0
            _target_weightedMAPE_call :float = 0.0
            for i, sr in _targetDF.iterrows():
                _c_t :float = sr["#Call"]
                functionName :str = sr["Name"]

                _A_t_resVar :float = sr[self.resVar]
                _A_t_resVarPerCall :float = sr[self.resVarPerCall]
                _A_t_call :float = sr["#Call"]

                _F_t_resVar :float = self.dict_resVar[functionName].subs(_target_env).evalf()
                _F_t_resVarPerCall :float = self.dict_resVarPerCall[functionName].subs(_target_env).evalf()
                _F_t_call :float = self.dict_call[functionName].subs(_target_env).evalf()

                
                _target_weightedMAPE_resVar += abs(_A_t_resVar - _F_t_resVar)/_A_t_resVar
                _target_weightedMAPE_resVarPerCall += abs(_A_t_resVarPerCall - _F_t_resVarPerCall)/_A_t_resVarPerCall
                _target_weightedMAPE_call += abs(_A_t_call - _F_t_call)/_A_t_call

            _target_weightedMAPE_resVar *= 100/_c_sum
            _target_weightedMAPE_resVarPerCall *= 100/_c_sum
            _target_weightedMAPE_call *= 100/_c_sum
            
            _series[f"weightedMAPE_{self.resVar}"] = _target_weightedMAPE_resVar
            _series[f"weightedMAPE_{self.resVarPerCall}"] = _target_weightedMAPE_resVarPerCall
            _series[f"weightedMAPE_call"] = _target_weightedMAPE_call

            _list_series.append(_series)
        
        self.DF_weightedMAPE :pd.DataFrame = pd.DataFrame(data=_list_series)
        return self.DF_weightedMAPE
                

In [12]:


dt_now = datetime.datetime.now()

benchmarkName :str = "mg"

mg_contents = ContentsForExtraP(trainDF = trainDF_MG, testDF= testDF_MG, resVar = "Exclusive", expVars= expVars_mg ,resVarPerCall = "ExclusivePerCall", benchmarkName = "mg")
mg_contents.build_all_models()
mg_contents.predict_train()
mg_contents.predict_test()
mg_適合度 :pd.DataFrame = mg_contents.get_DF_all_fitness_with_relativeErrorRate()
mg_重み付きMAPE :pd.DataFrame = mg_contents.get_DF_weightedMAPE()

outputFilePath :str = f"./outputs/{benchmarkName}_{dt_now.strftime('%Y%m%d%H%M%S')}.xlsx"
with pd.ExcelWriter(
    outputFilePath,
    engine="xlsxwriter",
    engine_kwargs={'options': {'strings_to_numbers': True}},
    ) as writer:
    mg_適合度.to_excel(writer, sheet_name="適合度"),
    mg_重み付きMAPE.to_excel(writer, sheet_name="重み付きMAPE"),


In [13]:


dt_now = datetime.datetime.now()

benchmarkName :str = "ep"

ep_contents = ContentsForExtraP(trainDF = trainDF_EP, testDF= testDF_EP, resVar = "Exclusive", expVars= expVars_ep ,resVarPerCall = "ExclusivePerCall", benchmarkName = benchmarkName)
ep_contents.build_all_models()
ep_contents.predict_train()
ep_contents.predict_test()
ep_適合度 :pd.DataFrame = ep_contents.get_DF_all_fitness_with_relativeErrorRate()
ep_重み付きMAPE :pd.DataFrame = ep_contents.get_DF_weightedMAPE()

outputFilePath :str = f"./outputs/{benchmarkName}_{dt_now.strftime('%Y%m%d%H%M%S')}.xlsx"
with pd.ExcelWriter(
    outputFilePath,
    engine="xlsxwriter",
    engine_kwargs={'options': {'strings_to_numbers': True}},
    ) as writer:
    ep_適合度.to_excel(writer, sheet_name="適合度"),
    ep_重み付きMAPE.to_excel(writer, sheet_name="重み付きMAPE"),


filePath = './extra-p_docker/share/input_ep_Exclusive@-TAU_application.txt'
modelerName = 'multi-parameter'
modelerOption = ' --options \\#spm=Basic \\#spo=poly_exponents=-1,0,1,2,3,log_exponents=0,1,force_combination_exponents=1,allow_negative_exponents=True'
str_ExtraPinputData = 'PARAMETER process\nPARAMETER size\n\nPOINTS ( 4 S )\nPOINTS ( 4 W )\nPOINTS ( 4 A )\nPOINTS ( 4 B )\nPOINTS ( 4 C )\nPOINTS ( 8 S )\nPOINTS ( 8 W )\nPOINTS ( 8 A )\nPOINTS ( 8 B )\nPOINTS ( 8 C )\nPOINTS ( 16 S )\nPOINTS ( 16 W )\nPOINTS ( 16 A )\nPOINTS ( 16 B )\nPOINTS ( 16 C )\nPOINTS ( 32 S )\nPOINTS ( 32 W )\nPOINTS ( 32 A )\nPOINTS ( 32 B )\nPOINTS ( 32 C )\nPOINTS ( 64 S )\nPOINTS ( 64 W )\nPOINTS ( 64 A )\nPOINTS ( 64 B )\nPOINTS ( 64 C )\n\nREGION reg\nMETRIC time\n\nDATA 7.75e-06 7.75e-06 7.75e-06\nDATA 7.499999999999999e-06 7.499999999999999e-06 7.499999999999999e-06\nDATA 7e-06 7e-06 7e-06\nDATA 7e-06 7e-06 7e-06\nDATA 7.166666666666667e-06 7.166666666666667e-06 7.166666666666667e-06\nDATA 7.499

SympifyError: Sympify of expression 'could not parse ''' failed, because of exception being raised:
SyntaxError: invalid syntax (<string>, line 0)

In [ ]:
dt_now = datetime.datetime.now()

benchmarkName :str = "ft"

ft_contents = ContentsForExtraP(trainDF = trainDF_FT, testDF= testDF_FT, resVar = "Exclusive", expVars= expVars_ft ,resVarPerCall = "ExclusivePerCall", benchmarkName = benchmarkName)
ft_contents.build_all_models()
ft_contents.predict_train()
ft_contents.predict_test()
ft_適合度 :pd.DataFrame = ft_contents.get_DF_all_fitness_with_relativeErrorRate()
ft_重み付きMAPE :pd.DataFrame = ft_contents.get_DF_weightedMAPE()

outputFilePath :str = f"./outputs/{benchmarkName}_{dt_now.strftime('%Y%m%d%H%M%S')}.xlsx"
with pd.ExcelWriter(
    outputFilePath,
    engine="xlsxwriter",
    engine_kwargs={'options': {'strings_to_numbers': True}},
    ) as writer:
    ft_適合度.to_excel(writer, sheet_name="適合度"),
    ft_重み付きMAPE.to_excel(writer, sheet_name="重み付きMAPE"),


In [ ]:
dt_now = datetime.datetime.now()

benchmarkName :str = "is"

is_contents = ContentsForExtraP(trainDF = trainDF_IS, testDF= testDF_IS, resVar = "Exclusive", expVars= expVars_is, resVarPerCall = "ExclusivePerCall", benchmarkName = benchmarkName)
is_contents.build_all_models()
is_contents.predict_train()
is_contents.predict_test()
is_適合度 :pd.DataFrame = is_contents.get_DF_all_fitness_with_relativeErrorRate()
is_重み付きMAPE :pd.DataFrame = is_contents.get_DF_weightedMAPE()

outputFilePath :str = f"./outputs/{benchmarkName}_{dt_now.strftime('%Y%m%d%H%M%S')}.xlsx"
with pd.ExcelWriter(
    outputFilePath,
    engine="xlsxwriter",
    engine_kwargs={'options': {'strings_to_numbers': True}},
    ) as writer:
    is_適合度.to_excel(writer, sheet_name="適合度"),
    is_重み付きMAPE.to_excel(writer, sheet_name="重み付きMAPE"),


In [ ]:
ep_適合度.mean(numeric_only=True)
ft_適合度.mean(numeric_only=True)
is_適合度.mean(numeric_only=True)
mg_適合度.mean(numeric_only=True)